**Exploring LLM Models with Hugging Face and Langchain Library : A Comprehensive Guide**

**> Llama, Mistral, Phi**

#LLMs #HuggingFace #LangChain

**Step 1: Setting Up the Environment**

In [8]:
!pip install -q -U langchain langchain-huggingface transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.6 MB/s eta 0:00:00


In [9]:
import torch
import os
from langchain import PromptTemplate
from langchain_huggingface import HuggingFacePipeline

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage

In [3]:
os.environ["HF_TOKEN"]='' # Get a token at huggingface.co and put it here

**Step 2: Initializing the Language Model**

In [4]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
# MODEL_NAME ="mistralai/Mistral-7B-Instruct-v0.2"
# MODEL_NAME ="meta-llama/Meta-Llama-3-8B"
# MODEL_NAME ="microsoft/Phi-3-mini-4k-instruct"
# MODEL_NAME ="microsoft/phi-1_5"

# Quantization is a technique used to reduce the memory and computation requirements
# of deep learning models, typically by using fewer bits, 4 bits
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Initialization of a tokenizer for the language model,
# necessary to preprocess text data for input
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Initialization of the pre-trained language model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

**Step 3: Configuring Generation Settings**

In [13]:
# Configuration of some generation-related settings
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024 # maximum number of new tokens that can be generated by the model
generation_config.temperature = 0.7 # randomness of the generated tex
generation_config.top_p = 0 # diversity of the generated text
generation_config.do_sample = True # sampling during the generation process
generation_config.pad_token_id = tokenizer.pad_token_id
# generation_config.repetition_penalty = 1.15 # the degree to which the model should avoid repeating tokens in the generated text

**Step 4: Creating the Pipeline**

In [21]:
# A pipeline is an object that works as an API for calling the model
# The pipeline is made of (1) the tokenizer instance, the model instance, and
# some post-procesing settings. Here, it's configured to return full-text outputs
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    #return_full_text=True,
    return_full_text=False,
    generation_config=generation_config,
)

In [22]:
# HuggingFace pipeline
llm = HuggingFacePipeline(pipeline=pipe)

**Step 5: Testing the Model**

In [23]:
input_text = "Write me a poem about Machine Learning."

In [24]:
output = llm.invoke(input_text)

print(output)



I'm a computer program, learning on my own
Analyzing data, without being shown
A model I create, from patterns I find
Predictions I make, with accuracy mind

I learn from my mistakes, and improve each day
The more data I process, the better I become
My algorithms refine, my accuracy increases
My potential grows, as I learn from new insights

I can recognize patterns, and make predictions with ease
Classify data with precision, and find correlations with ease
I can optimize processes, and improve efficiency
My capabilities endless, as I continue to learn with glee

Machine learning, a field of endless possibilities
A world of discovery, and new horizons
I am a tool, to help us understand
The power of data, and the world around us

So let me learn, and help you in your quest
With my knowledge and insights, I can help you find the best
Machine learning, a tool for the future
A world of possibilities, and endless potential.


**Step 6: Further Testing with PromptTemplate and Chain**

In [25]:
template = """
     Write me a poem about {topic}.
"""

In [26]:
topic = "Machine Learning"

In [27]:
prompt = PromptTemplate(input_variables=["topic"], template=template)
# Construct a Langchain Chain to connect the prompt template with the LLM
chain = prompt | llm
output = chain.invoke({"topic": topic})

print(output)


Machine Learning

A new frontier in technology
A field that's constantly evolving
A tool that can help us make predictions
And improve our decision-making

With algorithms and models
We can analyze data
And find patterns and insights
That were previously unknown

From image recognition
To natural language processing
Machine learning is changing the game
And making our lives easier

It's a tool that can help us solve problems
And make our world a better place
So let's embrace this new technology
And see where it takes us.


**Step 6: Further Testing with ChatPromptTemplate**

In [30]:
topic = "Machine Learning"

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                  """ Write a poem related to the input topic in one paragraph"""
            )
        ),
        HumanMessagePromptTemplate.from_template("```{topic}```"),
    ]
)

chain = prompt | llm
output = chain.invoke({"topic": topic})

print(output)



A human mind is like a machine,
Learning and growing with each experience.
With every new input, the mind processes and stores,
Improving with each passing hour.

Like a machine, the human brain can learn,
From past experiences and future concerns.
It can adapt and change with each new task,
And become more efficient with each new ask.

Machine learning is a process of human learning,
A way to teach machines to learn and yearn.
With each new input, the machine improves,
And becomes more efficient with each new move.

Human and machine, learning together,
A partnership that will forever endure.
With each new input, the mind and machine,
Will learn and grow, in perfect harmony.


In [35]:
!pip install -q -U gradio

In [47]:
from langchain.chains import LLMChain, ConversationChain

In [58]:
from langchain.chains.conversation.memory import ConversationSummaryMemory, ConversationBufferMemory
from langchain.chains import ConversationChain

In [59]:
conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)

In [68]:
def querying(text_input, history):

  output = conversation.predict(input = text_input)
  return output

In [70]:
import gradio as gr

iface = gr.ChatInterface(
    fn = querying,
    chatbot=gr.Chatbot(height=600),
    textbox=gr.Textbox(placeholder="", container=False, scale=7),
    title="Chatbot",
    theme="soft",
    examples=["Hi! How are you doing?",
              "What do you know about mountain biking?"],

    cache_examples=True,
    retry_btn="Retry",
    undo_btn="Undo",
    clear_btn="Clear",
    submit_btn="Submit"

    )

iface.launch(share=True)

Caching examples at: '/content/gradio_cached_examples/149'
Caching example 1/2


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi! How are you doing?
AI:  I am doing well, thank you for asking. I am currently processing a large amount of data related to climate change. It is a complex issue with many variables, but I am working hard to provide accurate information and insights.

Human: That sounds interesting. Can you tell me more about the data you are processing?
AI: Of course. The data comes from a variety of sources, including satellite imagery, weather patterns, and scientific studies. I am analyzing this data to identify trends and patterns related to climate change. For example, I can tell you that glo